In [ ]:
!pip install langchain

In [ ]:
# openai 설치
!pip install -qU langchain-openai

In [15]:
import getpass
import os

os.environ["OPENAI_API_KEY"]
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the follwing from English into Italian"),
    HumanMessage(content="hi!"),
]
model.invoke(messages)

In [ ]:
# OutputParsers - 사람에게 필요한 데이터만 반환

from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

result = model.invoke(messages)
parser.invoke(result)

In [ ]:
 # Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following into {language}:"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

result = prompt_template.inovke({"language": "italian", "text": "hi"})
result

result.to_messages()

In [ ]:
!pip install langchain langchain-chroma langchain-openai

In [9]:
from langchain_core.documents import Document

documents = [
   Document (
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document (
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document (
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document (
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document (
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    )
]

In [ ]:
# Vectorstore

from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(
    documents,
    embedding=OpenAIEmbeddings(),
)

In [ ]:
vectorstore.similarity_search("cat")

In [ ]:
# async
await vectorstore.asimilarity_search("cat")

In [ ]:
# score로 유사도 검사
vectorstore.similarity_search_with_score("cat")

In [ ]:
# 사용자 쿼리의 유사도 검사
embedding = OpenAIEmbeddings().embed_query("cat")

vectorstore.similarity_search_by_vector(embedding)

In [ ]:
# Retrievers
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1) # 가장 위의 값
# 위의 방식과 동일
# retriever = vectorstore.as_retriever(
#     search_type="similarity",
#     search_kwargs={"k": 1},
# )

# 여러 키워드 검색
retriever.batch(["cat", "shark"])

In [22]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
import getpass
import os
from langchain_openai import ChatOpenAI
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(model="gpt-3.5-turbo")

vectorstore = Chroma.from_documents(
    documents,
    embedding=OpenAIEmbeddings(),
)

retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

message = """
Answer this question using the provieded context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

response = rag_chain.invoke("tell me about cats")

response.content

'Cats are independent pets that often enjoy their own space.'

Tavily - key  발급 필요
- LLM과 RAG 시스템에 최적화된 검색엔진
- AI 최적화 : LLM과 원활한 통합을 위해 설계
- 최신 정보 : 실시간으로 업데이트되는 정보 제공
- 다양한 검색 모드 : 일반검색, 뉴스검색, 이미지 검색
- 콘텐츠 필터링 : 신뢰할 수 있는 소스의 정보만 제공
- 무료 사용량 : 월 1,000회 API 호출 제공

In [23]:
# Agent
%pip install -U langchain-community langgraph langchain-anthropic tavily-python

  Using cached pydantic_settings-2.7.0-py3-none-any.whl.metadata (3.5 kB)
  Using cached mypy_extensions-1.0.0-py3-none-any.whl.metadata (1.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.9 MB/s eta 0:00:00
Using cached pydantic_settings-2.7.0-py3-none-any.whl (29 kB)
Using cached mypy_extensions-1.0.0-py3-none-any.whl (4.7 kB)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [24]:
# Tavily로 실시간 날씨 정보 가져오기 
from langchain_community.tools.tavily_search import TavilySearchResults
import os
from dotenv import load_dotenv

load_dotenv()

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)

tools = [search]


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1735181669, 'localtime': '2024-12-25 18:54'}, 'current': {'last_updated_epoch': 1735181100, 'last_updated': '2024-12-25 18:45', 'temp_c': 9.5, 'temp_f': 49.0, 'is_day': 0, 'condition': {'text': 'Fog', 'icon': '//cdn.weatherapi.com/weather/64x64/night/248.png', 'code': 1135}, 'wind_mph': 6.5, 'wind_kph': 10.4, 'wind_degree': 275, 'wind_dir': 'W', 'pressure_mb': 1023.0, 'pressure_in': 30.2, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 97, 'cloud': 100, 'feelslike_c': 8.9, 'feelslike_f': 48.0, 'windchill_c': 8.9, 'windchill_f': 48.0, 'heatindex_c': 9.5, 'heatindex_f': 49.0, 'dewpoint_c': 9.1, 'dewpoint_f': 48.4, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 0.0, 'gust_mph': 9.4, 'gust_kph': 15.1}}"}, {'url': 'https://weathershogun.com/weather/usa/ca/san-fran

In [26]:
import getpass
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import create_react_agent # agent 



load_dotenv()
model = ChatOpenAI(model="gpt-4")

search = TavilySearchResults(max_results=2)
tools = [search]


# response = model.invoke([HumanMessage(content="hi!")])
# response.content
model_with_tools = model.bind_tools(tools)

# response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

agent_executor = create_react_agent(model, tools)

response = agent_executor.invoke({"messages": [HumanMessage(content="What's the weather in SF?")]})
response["messages"]


[HumanMessage(content="What's the weather in SF?", additional_kwargs={}, response_metadata={}, id='e3685343-6b2f-4c62-8bb3-cb3dc1c6d266'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_LWHYXX2iz74N0mQrfIwiVDd2', 'function': {'arguments': '{\n  "query": "current weather in San Francisco"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 88, 'total_tokens': 112, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-a66d64cb-3a1f-49a3-9499-5dc19934d4d1-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_LWHYXX2iz74N0mQrfIwiVDd